In [1]:
# Retrieve amount of sugar (in grams/100g of product)
# from database in FoodData Central using its API
import urllib.request, urllib.parse, urllib.error
import json
import ssl
import sqlite3

serviceurl = 'https://api.nal.usda.gov/fdc/v1/'

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

conn = sqlite3.connect('fooddata.sqlite')
cur = conn.cursor()

cur.execute('''
            CREATE TABLE IF NOT EXISTS Food
            (id INTEGER PRIMARY KEY UNIQUE, name TEXT UNIQUE, sugar FLOAT)''')
food = input('Enter food: ')

parms = dict()
parms['query'] = food
# Warnign: change YOUR_KEY on the following line
url = serviceurl + 'foods/search?' + urllib.parse.urlencode(parms) + '&api_key=YOUR_KEY'
# YOUR_KEY is the key provided when you request on https://fdc.nal.usda.gov/api-key-signup.html

#print('Retrieving', url)
uh = urllib.request.urlopen(url, context=ctx)
data = uh.read().decode()
#print('Retrieved', len(data), 'characters')

try:
    js = json.loads(data)
except:
    js = None
# As there are many products from one keyword I concentrate on the raw product
# Still, even with 'raw' in the name, there are variations. For eg.:
# search for keyword: 'banana' gives descriptions: 1)banana, raw; 2)pepper, banana, raw, etc.
# so I put the length of the description in a list and find the index of the min value
# (get the most basic description). Then I print the value (total sugar) of the same index
 
len_description = list() 
value_list = list()

for item in js['foods']: # js is a dictionary
    if ', raw' in item['lowercaseDescription']: # find only raw items
        for item2 in item['foodNutrients']:
            if item2['nutrientName'] == 'Sugars, total including NLEA':
                len_description.append(len(item['description'])) # keeps track of size of description
                value_list.append(item2['value'])
# In the case that there is no 'raw' product in the search                
if len(len_description) == 0:
    total_sugar = list()
    for item in js['foods']:
        if item['lowercaseDescription']==food: # matches search with item description
#            print(item['description'])
            for item2 in item['foodNutrients']:
                if item2['nutrientName'] == 'Sugars, total including NLEA':
                    total_sugar.append(item2['value'])
                    #print(item['description'])
                    break
    sugar_amount = f'{sum(total_sugar)/len(total_sugar):.2f}'# calculate an average with all
# products found and displays only 2 decimal points
    cur.execute('''INSERT OR IGNORE INTO Food (name, sugar) 
            VALUES (?, ?)''', (food, sugar_amount))
    print('Total sugar (g) in 100g:', sugar_amount)

else:
    index_min = min(range(len(len_description)), key=len_description.__getitem__) # finds index of shortest description
    cur.execute('''INSERT OR IGNORE INTO Food (name, sugar) 
            VALUES (?, ?)''', (food+', raw', value_list[index_min]))
    print('Total sugar (g) in 100g of RAW product:', value_list[index_min])
conn.commit()
cur.close()


Enter food: 


HTTPError: HTTP Error 403: Forbidden